# DAMA50: Mathematics for machine learning

## Written assignment 5

### Georgios Zavlanis


### Problem 7

a. First, we have to "translate" the problem into a linear programming problem. The process can be seen below:

We have 42 million dollars in total, for three types of investments.
- Let be $x_1$ for treasury notes, $x_2$ for bonds and $x_3$ for stocks.
- We have a minimum investment of 3 million dollars in each category, which means that $ x_1, x_2, x_3 \ge 3000000 $.
- Also, at least half of the total funds allocated to treasury notes and bonds, which means that $ x_1+x_2 \ge 21000000 $.
- Finally, the investment in bonds must not exceed twice the investment in treasury notes, which means that $ x_2 \le 2x_1 $ or $ x_2 - 2x_1 \le 0 $.

We need to maximize the returns, so we need max $ 0.07x_1 + 0.09x_2 + 0.1x_3 $.

b. So now, we are ready to solve the linear problem:

In [52]:
# Define the Linear Programming Problem (LP)
p=MixedIntegerLinearProgram(maximization = True)
v = p.new_variable(real=True, nonnegative = True)
x, y, z = v['x'], v['y'], v['z']

# Set the cost function
p.set_objective(0.07*x + 0.09*y + 0.1*z)

# and the constraints
p.add_constraint(x + y + z == 42000000)
p.add_constraint(x + y >= 21000000)
p.add_constraint(2*x >= y)
p.add_constraint(x >= 3000000)
p.add_constraint(y >= 3000000)
p.add_constraint(z >= 3000000)

# Solve the LP problem
p.solve()

3850000.0

In [57]:
p.get_values(x),p.get_values(y),p.get_values(z)

(7000000.0, 14000000.0, 21000000.0)

### Problem 8

First, let's create our function:

In [75]:
%display latex
x1, x2 = var('x1, x2')
f(x1, x1) = exp(x1)*(4*x1^2 + 2*x2^2 + 4*x1*x2 + 2*x2 + 1); f(x1, x1)

(4*x1^2 + 4*x1*x2 + 2*x2^2 + 2*x2 + 1)*e^x1

a. To find the stationary points using the analytically method, we follow the process below:

In [76]:
# Find the gradient
gradient = f(x1, x2).gradient();gradient

((10*x2^2 + 2*x2 + 1)*e^x2 + 2*(10*x2 + 1)*e^x2)

In [77]:
# Set gradient equals with zero
s = solve([gradient[0] == 0], [x1, x1]);s

[]

In [78]:
# Fid the Hessian matrix:
hessian = f(x1, x2).hessian();hessian

[(10*x2^2 + 2*x2 + 1)*e^x2 + 4*(10*x2 + 1)*e^x2 + 20*e^x2]

TypeError: cannot evaluate symbolic expression numerically